In [1]:
import string,re
import pandas as pd
import contractions
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from nltk.corpus import stopwords
from ipynb.fs.defs.cleaning import cleaned_text

pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_rows', 1000) 

In [2]:
df_labelled = pd.read_csv("./dataset/vax_tweets_sentiment_cleaned.csv")

In [3]:
df_latest = pd.read_csv("./dataset/vaccination_all_tweets.csv")

In [4]:
df_latest.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radio Network. Weeknights 7 Pacific-10 Eastern - Email comments/ideas to charles@charlesadlertonight.ca",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12 20:23:59,"Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternative news source from citizen journalists that haven't sold out. Real news & real views,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech… https://t.co/KxbSRoBEHq,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [5]:
df_latest.isnull().sum()

id                      0
user_name               2
user_location       66911
user_description    17018
user_created            0
user_followers          0
user_friends            0
user_favourites         0
user_verified           0
date                    0
text                    0
hashtags            49703
source                119
retweets                0
favorites               0
is_retweet              0
dtype: int64

In [6]:
df_latest.shape

(228207, 16)

In [7]:
df_labelled.shape

(26713, 6)

In [8]:
df_latest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228207 entries, 0 to 228206
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                228207 non-null  int64 
 1   user_name         228205 non-null  object
 2   user_location     161296 non-null  object
 3   user_description  211189 non-null  object
 4   user_created      228207 non-null  object
 5   user_followers    228207 non-null  int64 
 6   user_friends      228207 non-null  int64 
 7   user_favourites   228207 non-null  int64 
 8   user_verified     228207 non-null  bool  
 9   date              228207 non-null  object
 10  text              228207 non-null  object
 11  hashtags          178504 non-null  object
 12  source            228088 non-null  object
 13  retweets          228207 non-null  int64 
 14  favorites         228207 non-null  int64 
 15  is_retweet        228207 non-null  bool  
dtypes: bool(2), int64(6), object(8)
memory

In [9]:
print(df_latest['date'].min())
print(df_latest['date'].max())

2020-12-12 11:55:28
2021-11-23 20:58:08


In [10]:
df_latest['date'] = pd.to_datetime(df_latest['date'])
df_latest['date'] = df_latest['date'].dt.date

In [11]:
print(df_latest['date'].min())
print(df_latest['date'].max())

2020-12-12
2021-11-23


In [12]:
df_labelled['date'] = pd.to_datetime(df_labelled['date'])
df_labelled['date'] = df_labelled['date'].dt.date

In [13]:
print(df_labelled['date'].min())
print(df_labelled['date'].max())

2020-12-12
2021-03-16


In [14]:
df_latest_filtered = df_latest.loc[(df_latest['date'] > df_labelled['date'].max())]

In [15]:
print(df_latest_filtered['date'].min())
print(df_latest_filtered['date'].max())

2021-03-17
2021-11-23


In [16]:
df_latest_filtered.shape

(197643, 16)

In [17]:
#Find duplicated tweets from the same username
df_latest_filtered[df_latest_filtered.duplicated(['text','user_name'],keep = False)].sort_values(by ='text' )

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
203106,1448430851763552261,FinancialXpress,"New Delhi, India","A complete up-to-date source for biz news, #finance news, #stockmarket news, economy news online. \nहिंदी में @fehindi | https://t.co/yeMh2f2ymw",2009-06-24 11:29:19,761897,31,114,True,2021-10-13,#Covaxin is the first vaccine in the country for children below 12 years. https://t.co/XNuT1Q670H,['Covaxin'],TweetDeck,0,4,False
203453,1448138196013907973,FinancialXpress,"New Delhi, India","A complete up-to-date source for biz news, #finance news, #stockmarket news, economy news online. \nहिंदी में @fehindi | https://t.co/yeMh2f2ymw",2009-06-24 11:29:19,761897,31,114,True,2021-10-13,#Covaxin is the first vaccine in the country for children below 12 years. https://t.co/XNuT1Q670H,['Covaxin'],Twitter Web App,4,9,False
85824,1398783713933025280,FinancialXpress,"New Delhi, India","A complete up-to-date source for biz news, #finance news, #stockmarket news, economy news online. \nहिंदी में @fehindi | https://t.co/yeMh2f2ymw",2009-06-24 11:29:19,747052,37,26,True,2021-05-29,#Covaxin production will be doubled by May-June 2021 and then increased nearly sevenfold by July-August. https://t.co/m3jftFhB3i,['Covaxin'],TweetDeck,5,24,False
86225,1398508497797484546,FinancialXpress,"New Delhi, India","A complete up-to-date source for biz news, #finance news, #stockmarket news, economy news online. \nहिंदी में @fehindi | https://t.co/yeMh2f2ymw",2009-06-24 11:29:19,747052,37,26,True,2021-05-29,#Covaxin production will be doubled by May-June 2021 and then increased nearly sevenfold by July-August. https://t.co/m3jftFhB3i,['Covaxin'],Twitter Web App,6,46,False
156589,1423140772551925763,Hindustan Times,India,One of India's largest media companies. Latest news from around the world. Retweets are not endorsements,2009-04-29 10:11:34,8154562,140,2925,True,2021-08-05,"#Covaxin receives Good Manufacturing Practices (GMP) compliance certificate from Hungary, Bharat Biotech said \n\nhttps://t.co/WGCWu36zcS",['Covaxin'],Twitter Web App,3,56,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99386,1403235812565655554,Hindustan Times,India,One of India's largest media companies. Latest news from around the world. Retweets are not endorsements,2009-04-29 10:11:34,8065911,132,2929,True,2021-06-11,Why was #Covaxin rejected by the Food and Drug Administration (FDA)?\n\nAll you need to know\nhttps://t.co/NWvD4AJwWW,['Covaxin'],Twitter Web App,15,44,False
98636,1403441911231246338,Hindustan Times,India,One of India's largest media companies. Latest news from around the world. Retweets are not endorsements,2009-04-29 10:11:34,8065910,132,2929,True,2021-06-11,Why was #Covaxin rejected by the Food and Drug Administration (FDA)?\n\nAll you need to know\nhttps://t.co/NWvD4AJwWW,['Covaxin'],TweetDeck,18,70,False
98840,1403358862208503815,Hindustan Times,India,One of India's largest media companies. Latest news from around the world. Retweets are not endorsements,2009-04-29 10:11:34,8065910,132,2929,True,2021-06-11,Why was #Covaxin rejected by the Food and Drug Administration (FDA)?\n\nAll you need to know\nhttps://t.co/NWvD4AJwWW,['Covaxin'],TweetDeck,7,27,False
186267,1437400508998823941,Humair Siddique,NaN,NaN,2015-08-06 07:11:02,127,219,734,False,2021-09-13,https://t.co/kTx3ymAy6D\n#Rajkot #Pegasus #COVAXIN \nPLEASE VISIT MY WEBSITE,"['Rajkot', 'Pegasus', 'COVAXIN']",Twitter Web App,0,0,False


In [18]:
#Drop duplicated tweets
df_latest_filtered = df_latest_filtered.drop_duplicates('text')
df_latest_filtered.reset_index(drop=True, inplace=True)

In [19]:
df_latest_filtered.shape

(195838, 16)

In [20]:
#Copy original text and sentiment column into new dataframe
df_latest_filtered_preprocess = pd.DataFrame()
df_latest_filtered_preprocess['orig_text'] = df_latest_filtered['text']
df_latest_filtered_preprocess['user_name'] = df_latest_filtered['user_name']
df_latest_filtered_preprocess['date'] = df_latest_filtered['date']
df_latest_filtered_preprocess['hashtags'] = df_latest_filtered['hashtags']

df_latest_filtered_preprocess.head(100)

,orig_text,user_name,date,hashtags
0,4 cases of anaphylaxis in Qld.. following #COVID19 vaccination\n\nwithin the last 48 hours\n\n1 from #PfizerBioNTech 💉… https://t.co/fEjzFfKEX2,Mark 🎧🎶😷🔴⚪️💙,2021-03-17,"['COVID19', 'PfizerBioNTech']"
1,Over 1.5 million doses of China's #Sinovac #COVID19vaccine arrived in #Uruguay on Tuesday.\n\nPresident… https://t.co/JljNQHiZ8q,China Plus News,2021-03-17,"['Sinovac', 'COVID19vaccine', 'Uruguay']"
2,#Uruguay on Tuesday received a second batch of over 1.5 million doses of #COVID19vaccines from Chinese pharmaceutic… https://t.co/4OojbiVAQQ,CCTV+,2021-03-17,"['Uruguay', 'COVID19vaccines']"
3,Bully #China's embassy in India has made it compulsory for Indians travelling to #China to be vaccinated with… https://t.co/Yj7R8gD6rX,anbudan BALA🎶எ.அ.பாலா,2021-03-17,"['China', 'China']"
4,@democracynow peddled stories that fed into Sinophobia for the last few years (Uyghur ‘genocide’ and Hong Kong cov… https://t.co/mDbHvAWX20,☭🦇Karl Darx🦇☭,2021-03-17,NaN
5,"#Zimbabwe on Tuesday received a second batch of 200,000 #vaccine doses donated by China, a separate consignment of… https://t.co/SuufH0D3Lo",China Daily Africa,2021-03-17,"['Zimbabwe', 'vaccine']"
6,#Sinovac vaccine rollout on March 18\n\nhttps://t.co/9vFs6rBWWF https://t.co/Dx7eHRIPg9,Herald Malaysia,2021-03-17,['Sinovac']
7,"Apparently, #China has distributed its #SinoVac #Covid19 vaccine patent-free, and the US and Switzerland have fough… https://t.co/OVPt0yqMN8",Iqbal Faizer,2021-03-17,"['China', 'SinoVac', 'Covid19']"
8,#CCP has stopped roll-out of #Sinovac within #China as it has been found to be unsafe and has led to the hospitalis… https://t.co/gHjnfhxmmy,Mark,2021-03-17,"['CCP', 'Sinovac', 'China']"
9,#Uruguay on Tuesday received a new batch of China’s #Sinovac #COVID19 vaccine. The country launched a vaccination c… https://t.co/LcE941RubU,"People's Daily, China",2021-03-17,"['Uruguay', 'Sinovac', 'COVID19']"


In [21]:
#Run cleaning function and copy cleanted text into cleaned_text column
df_latest_filtered_preprocess['cleaned_text'] = df_latest_filtered_preprocess['orig_text'].transform(cleaned_text)

In [22]:
df_latest_filtered_preprocess = df_latest_filtered_preprocess[['orig_text','cleaned_text',
                                                               'user_name','date','hashtags']]

In [23]:
df_latest_filtered_preprocess.head(100)

,orig_text,cleaned_text,user_name,date,hashtags
0,4 cases of anaphylaxis in Qld.. following #COVID19 vaccination\n\nwithin the last 48 hours\n\n1 from #PfizerBioNTech 💉… https://t.co/fEjzFfKEX2,cases of anaphylaxis in qld following vaccination within the last hours from,Mark 🎧🎶😷🔴⚪️💙,2021-03-17,"['COVID19', 'PfizerBioNTech']"
1,Over 1.5 million doses of China's #Sinovac #COVID19vaccine arrived in #Uruguay on Tuesday.\n\nPresident… https://t.co/JljNQHiZ8q,over million doses of chinas arrived in on tuesday president,China Plus News,2021-03-17,"['Sinovac', 'COVID19vaccine', 'Uruguay']"
2,#Uruguay on Tuesday received a second batch of over 1.5 million doses of #COVID19vaccines from Chinese pharmaceutic… https://t.co/4OojbiVAQQ,on tuesday received a second batch of over million doses of from chinese pharmaceutic,CCTV+,2021-03-17,"['Uruguay', 'COVID19vaccines']"
3,Bully #China's embassy in India has made it compulsory for Indians travelling to #China to be vaccinated with… https://t.co/Yj7R8gD6rX,bully s embassy in india has made it compulsory for indians travelling to to be vaccinated with,anbudan BALA🎶எ.அ.பாலா,2021-03-17,"['China', 'China']"
4,@democracynow peddled stories that fed into Sinophobia for the last few years (Uyghur ‘genocide’ and Hong Kong cov… https://t.co/mDbHvAWX20,peddled stories that fed into sinophobia for the last few years uyghur genocide and hong kong cov,☭🦇Karl Darx🦇☭,2021-03-17,NaN
5,"#Zimbabwe on Tuesday received a second batch of 200,000 #vaccine doses donated by China, a separate consignment of… https://t.co/SuufH0D3Lo",on tuesday received a second batch of doses donated by china a separate consignment of,China Daily Africa,2021-03-17,"['Zimbabwe', 'vaccine']"
6,#Sinovac vaccine rollout on March 18\n\nhttps://t.co/9vFs6rBWWF https://t.co/Dx7eHRIPg9,vaccine rollout on march,Herald Malaysia,2021-03-17,['Sinovac']
7,"Apparently, #China has distributed its #SinoVac #Covid19 vaccine patent-free, and the US and Switzerland have fough… https://t.co/OVPt0yqMN8",apparently has distributed its vaccine patent free and the us and switzerland have fough,Iqbal Faizer,2021-03-17,"['China', 'SinoVac', 'Covid19']"
8,#CCP has stopped roll-out of #Sinovac within #China as it has been found to be unsafe and has led to the hospitalis… https://t.co/gHjnfhxmmy,has stopped roll out of within as it has been found to be unsafe and has led to the hospitalis,Mark,2021-03-17,"['CCP', 'Sinovac', 'China']"
9,#Uruguay on Tuesday received a new batch of China’s #Sinovac #COVID19 vaccine. The country launched a vaccination c… https://t.co/LcE941RubU,on tuesday received a new batch of chinas vaccine the country launched a vaccination c,"People's Daily, China",2021-03-17,"['Uruguay', 'Sinovac', 'COVID19']"


In [24]:
#Define stop words and remove stop words function

def rm_stopwords(text):
    
    stop_words=['in','of','at','a','the','to','is','and','i','for','my','on','it','you','this','are','be','as',
           'an', 'all', 'if', 'our', 'ours', 'ourselves', 'was', 'that', 'that’s','there','me','they','them',
            'themselves','will','which','who','who’s','what','she','her','herself','he','him','his','himself',
            'we','by','those','these','your','yours','you’re','yourself', 'yourselves','or','other','off',
            'their','theirs','then','where','where’s','when','when’s','its','itself','so','onto','every',
            'each','been','being','having','here','here’s','well','say','says','said','why','how','i’m','whom',
            'us', 'myself','s','am','amp','have','has','had','with','do']
    
    cleaned_tokens = []
    for i in text.split():
        if len(i) > 1 and i not in stop_words:
            cleaned_tokens.append(i)
            
    return ' '.join(cleaned_tokens)

In [25]:
df_latest_filtered_preprocess['cleaned_text'] = df_latest_filtered_preprocess['cleaned_text'].transform(rm_stopwords)

In [26]:
#Remove 0 word tweets after preprocessed

null_cleanedtext = []

for a,b in zip(df_latest_filtered_preprocess.cleaned_text, df_latest_filtered_preprocess.index): 
    
    if len(a.split()) == 0:
        null_cleanedtext.append(b)

In [27]:
#List all 0 word tweets
df_latest_filtered_preprocess[df_latest_filtered_preprocess.index.isin(null_cleanedtext)]

,orig_text,cleaned_text,user_name,date,hashtags
115,@OmarOma64360171 @nmc820206 @SaShaLvatoRe @ChilpoEn @luizcosta76 @dr_coreas @diddieralferez @bernardogalue… https://t.co/M9MtXqYr0i,,Kwhusih,2021-03-17,NaN
613,#Covid_19 #Coronaviruspakistan #Coronaviruspakistan #NCOCPakistan #ncoc @Asad_Umar @fslsltn #Sinopharm #China… https://t.co/ts4J9fgWBo,,NCOC,2021-03-17,"['Covid_19', 'Coronaviruspakistan', 'Coronaviruspakistan', 'NCOCPakistan', 'ncoc', 'Sinopharm', 'China']"
644,#moderna #vaccinated #timetotravel #losangeles #goodbyecovid19 #dodgersstadium #franceherewecome #byrontravelgroup… https://t.co/0UTW6hZuBS,,Chad Rodarme,2021-03-18,"['moderna', 'vaccinated', 'timetotravel', 'losangeles', 'goodbyecovid19', 'dodgersstadium', 'franceherewecome', 'byrontravelgroup']"
793,@naomirwolf #Democracy #EME #GregPalast #DrWilliamHaseltine #Electo #CNN #JoeBiden #AOC #Everlane #PandemicResponse… https://t.co/1cJ69kidGk,,DailyClout,2021-03-17,"['Democracy', 'EME', 'GregPalast', 'DrWilliamHaseltine', 'Electo', 'CNN', 'JoeBiden', 'AOC', 'Everlane', 'PandemicResponse']"
1168,@nair_hena @sbk1963 @chimnibai @KishorTrivedi7 @sureshseshadri1 @Chouhan_ks @hpbindass @keeranl @Anju1951Purohit… https://t.co/zM2ldEwDX9,,ajitsinh Jagirdar 🇮🇳 #We_The_Threadeans,2021-03-17,NaN
...,...,...,...,...,...
195186,@BhavDhillonnz @chrishipkins @minhealthnz @AshBloomfield @jacindaardern @AndrewLittleMP @gmsharmanz @IndiainNZ… https://t.co/ylZBimxkbm,,Vikram Jog,2021-11-17,NaN
195222,#OCGN #Covaxin #Official_WSB #Webull #RobinhoodTraders #nurses #KABC #EyewitnessNews #LosAngeles #TelevisionNews… https://t.co/BjHi2raJnC,,wil,2021-11-16,"['OCGN', 'Covaxin', 'Official_WSB', 'Webull', 'RobinhoodTraders', 'nurses', 'KABC', 'EyewitnessNews', 'LosAngeles', 'TelevisionNews']"
195438,#CYTO #OCGN #Covaxin #Official_WSB #Webull #RobinhoodTraders #nurses #KABC #EyewitnessNews #TelevisionNews… https://t.co/YLw2TiWF7N,,wil,2021-11-15,"['CYTO', 'OCGN', 'Covaxin', 'Official_WSB', 'Webull', 'RobinhoodTraders', 'nurses', 'KABC', 'EyewitnessNews', 'TelevisionNews']"
195454,@mae_rusia @EmbRusiaEnArgEs @EmbRusUruguay @EmbRusiaMexico @sputnikvaccine @embajada_rusia @EmbRusPan… https://t.co/XpW38FDjKC,,Angie SB♛🇦🇷 🇮🇹,2021-11-23,NaN


In [28]:
print("Total tweets with 0 word: ",len(null_cleanedtext))
print("Current dataset shape: ",df_latest_filtered_preprocess.shape)

Total tweets with 0 word:  1547
Current dataset shape:  (195838, 5)


In [29]:
#Remove rows with tweets with 0 word
df_latest_filtered_preprocess = df_latest_filtered_preprocess[~df_latest_filtered_preprocess.index.isin(null_cleanedtext)]

print("Current dataset shape: ",df_latest_filtered_preprocess.shape)

df_latest_filtered_preprocess.head(10)

Current dataset shape:  (194291, 5)


,orig_text,cleaned_text,user_name,date,hashtags
0,4 cases of anaphylaxis in Qld.. following #COVID19 vaccination\n\nwithin the last 48 hours\n\n1 from #PfizerBioNTech 💉… https://t.co/fEjzFfKEX2,cases anaphylaxis qld following vaccination within last hours from,Mark 🎧🎶😷🔴⚪️💙,2021-03-17,"['COVID19', 'PfizerBioNTech']"
1,Over 1.5 million doses of China's #Sinovac #COVID19vaccine arrived in #Uruguay on Tuesday.\n\nPresident… https://t.co/JljNQHiZ8q,over million doses chinas arrived tuesday president,China Plus News,2021-03-17,"['Sinovac', 'COVID19vaccine', 'Uruguay']"
2,#Uruguay on Tuesday received a second batch of over 1.5 million doses of #COVID19vaccines from Chinese pharmaceutic… https://t.co/4OojbiVAQQ,tuesday received second batch over million doses from chinese pharmaceutic,CCTV+,2021-03-17,"['Uruguay', 'COVID19vaccines']"
3,Bully #China's embassy in India has made it compulsory for Indians travelling to #China to be vaccinated with… https://t.co/Yj7R8gD6rX,bully embassy india made compulsory indians travelling vaccinated,anbudan BALA🎶எ.அ.பாலா,2021-03-17,"['China', 'China']"
4,@democracynow peddled stories that fed into Sinophobia for the last few years (Uyghur ‘genocide’ and Hong Kong cov… https://t.co/mDbHvAWX20,peddled stories fed into sinophobia last few years uyghur genocide hong kong cov,☭🦇Karl Darx🦇☭,2021-03-17,NaN
5,"#Zimbabwe on Tuesday received a second batch of 200,000 #vaccine doses donated by China, a separate consignment of… https://t.co/SuufH0D3Lo",tuesday received second batch doses donated china separate consignment,China Daily Africa,2021-03-17,"['Zimbabwe', 'vaccine']"
6,#Sinovac vaccine rollout on March 18\n\nhttps://t.co/9vFs6rBWWF https://t.co/Dx7eHRIPg9,vaccine rollout march,Herald Malaysia,2021-03-17,['Sinovac']
7,"Apparently, #China has distributed its #SinoVac #Covid19 vaccine patent-free, and the US and Switzerland have fough… https://t.co/OVPt0yqMN8",apparently distributed vaccine patent free switzerland fough,Iqbal Faizer,2021-03-17,"['China', 'SinoVac', 'Covid19']"
8,#CCP has stopped roll-out of #Sinovac within #China as it has been found to be unsafe and has led to the hospitalis… https://t.co/gHjnfhxmmy,stopped roll out within found unsafe led hospitalis,Mark,2021-03-17,"['CCP', 'Sinovac', 'China']"
9,#Uruguay on Tuesday received a new batch of China’s #Sinovac #COVID19 vaccine. The country launched a vaccination c… https://t.co/LcE941RubU,tuesday received new batch chinas vaccine country launched vaccination,"People's Daily, China",2021-03-17,"['Uruguay', 'Sinovac', 'COVID19']"


In [30]:
#Find duplicated 'cleaned_text' from the same username
df_latest_filtered_preprocess[df_latest_filtered_preprocess.duplicated(['cleaned_text','user_name'],keep = False)].sort_values(by ='orig_text' )

,orig_text,cleaned_text,user_name,date,hashtags
44243,!!Banglore!!\nOxygen Beds Available\n.\n.\n#helpnagar #covid #covid19 #help #India #banglore #nonprofit #medical… https://t.co/Stn51MNxIj,banglore oxygen beds available,Help Nagar,2021-05-17,"['helpnagar', 'covid', 'covid19', 'help', 'India', 'banglore', 'nonprofit', 'medical']"
43690,!!Banglore!!\nOxygen Beds Available\n.\n.\n#helpnagar #covid #covid19 #help #India #banglore #nonprofit #medical… https://t.co/pmBAUCFlql,banglore oxygen beds available,Help Nagar,2021-05-18,"['helpnagar', 'covid', 'covid19', 'help', 'India', 'banglore', 'nonprofit', 'medical']"
44238,!!Delhi!!\nOxygen Beds Available\n.\n.\n#helpnagar #covid #covid19 #help #India #delhi #nonprofit #medical #urgent… https://t.co/OA050QHlKv,delhi oxygen beds available,Help Nagar,2021-05-17,"['helpnagar', 'covid', 'covid19', 'help', 'India', 'delhi', 'nonprofit', 'medical', 'urgent']"
44848,!!Delhi!!\nOxygen Beds Available\n.\n.\n#helpnagar #covid #covid19 #help #India #delhi #nonprofit #medical #urgent… https://t.co/ZnN9gRdRVD,delhi oxygen beds available,Help Nagar,2021-05-16,"['helpnagar', 'covid', 'covid19', 'help', 'India', 'delhi', 'nonprofit', 'medical', 'urgent']"
44993,!!Delhi!!\nOxygen Beds Available\n.\n.\n#helpnagar #covid #covid19 #help #India #delhi #nonprofit #medical #urgent… https://t.co/xF9RySpn2A,delhi oxygen beds available,Help Nagar,2021-05-16,"['helpnagar', 'covid', 'covid19', 'help', 'India', 'delhi', 'nonprofit', 'medical', 'urgent']"
...,...,...,...,...,...
169512,🛑👉🏽 Fuck #JoeBiden ! 👉🏽\n\n#AntiPass #antivax #macron #Manifs9octobre #HDpros2 #BFMTV #CNEWS #pfizer #Moderna #Vaccin… https://t.co/3xpV9QzFDy,fuck,BLF.TV,2021-10-10,"['JoeBiden', 'AntiPass', 'antivax', 'macron', 'Manifs9octobre', 'HDpros2', 'BFMTV', 'CNEWS', 'pfizer', 'Moderna', 'Vaccin']"
145141,🤦🏻‍♂️\nWe need #COVAXIN 🌏\n$OCGN #COVAXIN #OCUGEN #COVIDVACCINE #OCGN #BharatBiotech #Vaccigen #WHO @who @Ocugen… https://t.co/IL1nivuPkQ,need ocgn,Xukki🌍,2021-08-21,"['COVAXIN', 'COVAXIN', 'OCUGEN', 'COVIDVACCINE', 'OCGN', 'BharatBiotech', 'Vaccigen', 'WHO']"
116205,🥇@lifewatchpharmacy🥇 has you covered! #Peel #GTA @citymississauga @BonnieCrombie @OmarAlghabra @CBCAlerts… https://t.co/XeieKvtCX9,covered,Life Watch Pharmacy,2021-07-27,"['Peel', 'GTA']"
138713,🦠💉⚕️\n\nCovid vaccines still effective against Delta variant https://t.co/hQ0ysm8Oix via @BBCNews \n\n#COVID19 #COVID… https://t.co/OZjLXcU4mr,covid vaccines still effective against delta variant via,VUBBLE,2021-08-20,"['COVID19', 'COVID']"


In [31]:
#Drop duplicated tweets
df_latest_filtered_preprocess = df_latest_filtered_preprocess.drop_duplicates('cleaned_text')
df_latest_filtered_preprocess.reset_index(drop=True, inplace=True)

In [32]:
df_latest_filtered_preprocess.shape

(135298, 5)

In [33]:
df_latest_filtered_preprocess.head(100)

,orig_text,cleaned_text,user_name,date,hashtags
0,4 cases of anaphylaxis in Qld.. following #COVID19 vaccination\n\nwithin the last 48 hours\n\n1 from #PfizerBioNTech 💉… https://t.co/fEjzFfKEX2,cases anaphylaxis qld following vaccination within last hours from,Mark 🎧🎶😷🔴⚪️💙,2021-03-17,"['COVID19', 'PfizerBioNTech']"
1,Over 1.5 million doses of China's #Sinovac #COVID19vaccine arrived in #Uruguay on Tuesday.\n\nPresident… https://t.co/JljNQHiZ8q,over million doses chinas arrived tuesday president,China Plus News,2021-03-17,"['Sinovac', 'COVID19vaccine', 'Uruguay']"
2,#Uruguay on Tuesday received a second batch of over 1.5 million doses of #COVID19vaccines from Chinese pharmaceutic… https://t.co/4OojbiVAQQ,tuesday received second batch over million doses from chinese pharmaceutic,CCTV+,2021-03-17,"['Uruguay', 'COVID19vaccines']"
3,Bully #China's embassy in India has made it compulsory for Indians travelling to #China to be vaccinated with… https://t.co/Yj7R8gD6rX,bully embassy india made compulsory indians travelling vaccinated,anbudan BALA🎶எ.அ.பாலா,2021-03-17,"['China', 'China']"
4,@democracynow peddled stories that fed into Sinophobia for the last few years (Uyghur ‘genocide’ and Hong Kong cov… https://t.co/mDbHvAWX20,peddled stories fed into sinophobia last few years uyghur genocide hong kong cov,☭🦇Karl Darx🦇☭,2021-03-17,NaN
5,"#Zimbabwe on Tuesday received a second batch of 200,000 #vaccine doses donated by China, a separate consignment of… https://t.co/SuufH0D3Lo",tuesday received second batch doses donated china separate consignment,China Daily Africa,2021-03-17,"['Zimbabwe', 'vaccine']"
6,#Sinovac vaccine rollout on March 18\n\nhttps://t.co/9vFs6rBWWF https://t.co/Dx7eHRIPg9,vaccine rollout march,Herald Malaysia,2021-03-17,['Sinovac']
7,"Apparently, #China has distributed its #SinoVac #Covid19 vaccine patent-free, and the US and Switzerland have fough… https://t.co/OVPt0yqMN8",apparently distributed vaccine patent free switzerland fough,Iqbal Faizer,2021-03-17,"['China', 'SinoVac', 'Covid19']"
8,#CCP has stopped roll-out of #Sinovac within #China as it has been found to be unsafe and has led to the hospitalis… https://t.co/gHjnfhxmmy,stopped roll out within found unsafe led hospitalis,Mark,2021-03-17,"['CCP', 'Sinovac', 'China']"
9,#Uruguay on Tuesday received a new batch of China’s #Sinovac #COVID19 vaccine. The country launched a vaccination c… https://t.co/LcE941RubU,tuesday received new batch chinas vaccine country launched vaccination,"People's Daily, China",2021-03-17,"['Uruguay', 'Sinovac', 'COVID19']"


In [34]:
loaded_model = joblib.load('./modelling/model/LR_model.joblib')

In [35]:
def predict_dataset(text):
    return loaded_model.predict([text])[0]

df_latest_filtered_preprocess['sentiment'] = df_latest_filtered_preprocess['cleaned_text'].apply(predict_dataset)

In [36]:
df_latest_filtered_preprocess.head(100)

,orig_text,cleaned_text,user_name,date,hashtags,sentiment
0,4 cases of anaphylaxis in Qld.. following #COVID19 vaccination\n\nwithin the last 48 hours\n\n1 from #PfizerBioNTech 💉… https://t.co/fEjzFfKEX2,cases anaphylaxis qld following vaccination within last hours from,Mark 🎧🎶😷🔴⚪️💙,2021-03-17,"['COVID19', 'PfizerBioNTech']",negative
1,Over 1.5 million doses of China's #Sinovac #COVID19vaccine arrived in #Uruguay on Tuesday.\n\nPresident… https://t.co/JljNQHiZ8q,over million doses chinas arrived tuesday president,China Plus News,2021-03-17,"['Sinovac', 'COVID19vaccine', 'Uruguay']",neutral
2,#Uruguay on Tuesday received a second batch of over 1.5 million doses of #COVID19vaccines from Chinese pharmaceutic… https://t.co/4OojbiVAQQ,tuesday received second batch over million doses from chinese pharmaceutic,CCTV+,2021-03-17,"['Uruguay', 'COVID19vaccines']",neutral
3,Bully #China's embassy in India has made it compulsory for Indians travelling to #China to be vaccinated with… https://t.co/Yj7R8gD6rX,bully embassy india made compulsory indians travelling vaccinated,anbudan BALA🎶எ.அ.பாலா,2021-03-17,"['China', 'China']",neutral
4,@democracynow peddled stories that fed into Sinophobia for the last few years (Uyghur ‘genocide’ and Hong Kong cov… https://t.co/mDbHvAWX20,peddled stories fed into sinophobia last few years uyghur genocide hong kong cov,☭🦇Karl Darx🦇☭,2021-03-17,NaN,neutral
5,"#Zimbabwe on Tuesday received a second batch of 200,000 #vaccine doses donated by China, a separate consignment of… https://t.co/SuufH0D3Lo",tuesday received second batch doses donated china separate consignment,China Daily Africa,2021-03-17,"['Zimbabwe', 'vaccine']",neutral
6,#Sinovac vaccine rollout on March 18\n\nhttps://t.co/9vFs6rBWWF https://t.co/Dx7eHRIPg9,vaccine rollout march,Herald Malaysia,2021-03-17,['Sinovac'],neutral
7,"Apparently, #China has distributed its #SinoVac #Covid19 vaccine patent-free, and the US and Switzerland have fough… https://t.co/OVPt0yqMN8",apparently distributed vaccine patent free switzerland fough,Iqbal Faizer,2021-03-17,"['China', 'SinoVac', 'Covid19']",neutral
8,#CCP has stopped roll-out of #Sinovac within #China as it has been found to be unsafe and has led to the hospitalis… https://t.co/gHjnfhxmmy,stopped roll out within found unsafe led hospitalis,Mark,2021-03-17,"['CCP', 'Sinovac', 'China']",negative
9,#Uruguay on Tuesday received a new batch of China’s #Sinovac #COVID19 vaccine. The country launched a vaccination c… https://t.co/LcE941RubU,tuesday received new batch chinas vaccine country launched vaccination,"People's Daily, China",2021-03-17,"['Uruguay', 'Sinovac', 'COVID19']",neutral


In [37]:
df_latest_filtered_preprocess['sentiment'].value_counts()

neutral     91544
positive    24885
negative    18869
Name: sentiment, dtype: int64

In [38]:
df_latest_filtered_preprocess.isnull().sum()

orig_text           0
cleaned_text        0
user_name           1
date                0
hashtags        28188
sentiment           0
dtype: int64

In [39]:
#Export to pickle

df_latest_filtered_preprocess.to_pickle('./dataset/vax_tweets_sentiment_latest_labeled.pkl')